In [1]:
'''*** Sparse matrix multiplication ***'''
import numpy as np

A_real = np.array([3, 1.5, 2,4 ,2.2, 2, 6, 4, -9], dtype=np.float64)
I_row = np.array([0, 0, 1, 1, 1, 2, 2, 3, 3], dtype=np.int32)
I_col = np.array([0, 2, 0, 1, 2, 1, 2, 2, 3], dtype=np.int32) 

nonzero = len(A_real)
dim = 4
y = np.zeros(dim) + 1.
z = np.zeros(dim)

for k in range(nonzero):
    z[I_row[k]] = z[I_row[k]] + A_real[k] * y[I_col[k]]

print(z)

[ 4.5  8.2  8.  -5. ]


In [2]:
# copy from ws03
def upper_triangular_solve(A, b):
    """
    Solve the system  A x = b  where A is assumed to be lower triangular,
    i.e. A(i,j) = 0 for j > i, and the diagonal is assumed to be nonzero,
    i.e. A(i,i) != 0.
    
    The code checks that A is lower triangular and converts A and b to
    double precision before computing.

    ARGUMENTS:  A   lower triangular n x n array
                b   right hand side column n-vector

    RETURNS:    x   column n-vector solution
    """

    # we should take care to ensure that arrays are stored with the correct type - float!
    A = A.astype(np.float64)
    b = b.astype(np.float64)
      
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape=}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape=} {b.shape=}')
    
    # check A is upper triangular
    for i in range(n):
        for j in range(0,i):
            if not np.isclose(A[i, j], 0.0):
                raise ValueError(f'A is not upper triangular! {A[i, j]=}')

    # checks whether A has zero diagonal element
    for i in range(n):
        if np.isclose(A[i, i], 0.0):
            raise ValueError(f'A[{i}, {i}] is zero')
    
    #create a new array to store the results
    x = np.empty_like(b)
    
    # perform backwards subsitution
    x[n-1] = b[n-1] / A[n-1, n-1]
    for i in range(2,n+1):
        x[n-i] = b[n-i] / A[n-i, n-i]
        for j in range(n-i+1, n):
            x[n-i] = x[n-i] - A[n-i,j]*x[j] / A[n-i, n-i]
        
    return x

In [3]:
'''*** Gaussian elimination with pivoting ***'''
def Gaussian_elimination_pivoting(A, b, verbose=False):
    # To ensure that arrays are stored in double precision.
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # size of solution vector / the square matrix A
    n=len(b) # or   n, n = A.shape
        
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape=}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape=} {b.shape=}')
    
    if verbose:
        print('starting system\n', A, b)
  
    # perform forward elimination
    for i in range(n):          
        # find the index of the maximal value in column i on or below
        # the diagonal of A
        maximum = abs(A[i,i])
        max_index = i
        for j in range(i+1,n):
            if abs(A[j,i]) > maximum :
                maximum = abs(A[j,i])               
                max_index = j   
                                       
        
        # swap two max_indexs: i and max_index[i]
        temp = b[i]
        b[i] = b[max_index]
        b[max_index] = temp
        for j in range(n):
            temp = A[i,j]
            A[i,j] = A[max_index,j]
            A[max_index,j] = temp  
            
        
        # check diagonal
        if np.isclose(A[i, i], 0.0):
            raise ValueError(f'A has zero on diagonal! A[{i}, {i}] = 0') 

        # row j <- row j - (a_{ji} / a_{ii}) row i
        for j in range(i+1, n):
            if verbose:
                print(f'row {j} <- row {j} - {A[j, i] / A[i, i]} row {i}')
            factor = A[j, i] / A[i, i]
            for k in range(0, n):
                A[j, k] = A[j, k] - factor * A[i, k]
            b[j] = b[j] - factor * b[i]
        
    return upper_triangular_solve(A, b)

In [4]:
'''testing'''
A = np.array([[-10, 2, 0, 67], [-2, 50, -77, 1.e-5], [1, 7, 30, 8], [-10, -7, 0.001, 80]])
b = np.array([1, 2, 9, 0])

# numpy linear solvers
x0 = np.linalg.solve(A,b)
#x0 = np.linalg.inv(A).dot(b)
print("Solution by numpy solver:", x0)

x = Gaussian_elimination_pivoting(A, b)
print("Gaussian elimination: ",x)
print("Residual: ", np.matmul(A,x)-b)

# extra test to ensure we have solved the problem
np.testing.assert_almost_equal(np.linalg.norm(b - A @ x), 0.0)

Solution by numpy solver: [0.9244595  0.31826746 0.15668124 0.14340388]
Gaussian elimination:  [0.9244595  0.31826746 0.15668124 0.14340388]
Residual:  [ 0.00000000e+00 -1.77635684e-15 -1.77635684e-15  0.00000000e+00]


In [5]:
'''testing 2'''
n=20
A = np.random.rand(n, n)
b = np.random.rand(n)

# numpy linear solvers
x0 = np.linalg.solve(A,b)
#x0 = np.linalg.inv(A).dot(b)
print("Solution by numpy solver:", x0)

x = Gaussian_elimination_pivoting(A, b)
print("Gaussian elimination: ",x)
print("Residual: ", np.matmul(A,x)-b)

# extra test to ensure we have solved the problem
np.testing.assert_almost_equal(np.linalg.norm(b - A @ x), 0.0)

Solution by numpy solver: [ 0.07776123 -0.34595467  0.12073569 -2.67345581  4.68962512  0.17783951
 -1.19457174  0.67849765  3.88262769 -2.46816894 -2.70313618 -2.6112638
  1.46850641 -0.23041472  1.46813633  0.48901412 -3.17361453 -1.25750879
  2.22946942  1.9540522 ]
Gaussian elimination:  [ 0.07776123 -0.34595467  0.12073569 -2.67345581  4.68962512  0.17783951
 -1.19457174  0.67849765  3.88262769 -2.46816894 -2.70313618 -2.6112638
  1.46850641 -0.23041472  1.46813633  0.48901412 -3.17361453 -1.25750879
  2.22946942  1.9540522 ]
Residual:  [ 3.33066907e-16  1.22124533e-15  1.11022302e-16  4.44089210e-16
  8.88178420e-16  1.66533454e-15  0.00000000e+00 -1.66533454e-16
  1.11022302e-16  2.22044605e-16  4.44089210e-16 -8.88178420e-16
  3.33066907e-16  1.99840144e-15  1.44328993e-15  9.99200722e-16
  3.33066907e-16  3.33066907e-16  6.66133815e-16 -6.66133815e-16]
